In [1]:
%run library_imports.ipynb
%run function_imports.ipynb


<div class="alert alert-block alert-warning">
    <h1><b>Model Construction</b></h1> 
</div>

***

In [2]:
savedModels=[]
savedAnnotations={}
savedkfkr={}
savedtypes={}
savedSpeciesNoDuplicateSingleDict={}
initialValues={}
Nmatrices={}
savedpVals={}

equations={}
equations['Template 3'] = r"$V_{SS} = \frac{(p_{1} q_{Ao} - p_{2} q_{Ai})}{(p_{3} + p_{4} q_{Ao} + p_{5} q_{Ai} q_{Ao} + p_{6} q_{Ai})}$"
equations['Template 4'] = r"$V_{SS} = \frac{(p_{1} q_{Ao} q_{Bi} - p_{2} q_{Ai} q_{Bo})}{(p_{3} q_{Ao} + p_{4} q_{Ai} + p_{5} q_{Bo} + p_{6} q_{Bi} + p_{7} q_{Ao} q_{Bi} + p_{8} q_{Ai} q_{Ao} + p_{9} q_{Ai} q_{Bo} + p_{10} q_{Bo} q_{Bi})}$"
equations['Template 7'] = r"$V_{SS} = \frac{(p_{1} q_{Ai} q_{Bo} q_{Co} - p_{2} q_{Ao} q_{Bi} q_{Ci})}{(p_{3} q_{Ai} + p_{4} q_{Ao} + p_{5} q_{Bi} + p_{6} q_{Bo} + p_{7} q_{Ci} + p_{8} q_{Co} + p_{9} q_{Ai} q_{Ao} + p_{10} q_{Bi} q_{Bo} + p_{11} q_{Ci} q_{Co} + p_{12} q_{Ai} q_{Bo} q_{Co} + p_{13} q_{Bi} q_{Ci} q_{Ao} )}$"
equations['Template 11'] = r"$V_{SS} = \frac{(p_{1} q_{Ai} q_{Bo} q_{Co} q_{Do} - p_{2} q_{Ao} q_{Bi} q_{Ci} q_{Di})}{(p_{3} q_{Ai} + p_{4} q_{Ao} + p_{5} q_{Bi} + p_{6} q_{Bo} + p_{7} q_{Ci} + p_{8} q_{Co} + p_{9} q_{Di} + p_{10} q_{Do} + p{11} q_{Ai} q_{Ao} + p_{12} q_{Bi} q_{Bo} + p_{13} q_{Ci} q_{Co} + p_{14} q_{Di} q_{Do} + p_{15} q_{Ai} q_{Bo} q_{Co} q_{Do} + p_{16} q_{Bi} q_{Ci} q_{Ao} q_{Di})}$"
equations['Template 5'] = r"$V_{SS} = \frac{(p_{1} q_{Bo} q_{Ao} e^{(F V_m/R T)}-p_{2} q_{Ai} q_{Bi})}{(p_{3} q_{Bi} q_{Bo}+p_{4} q_{Bo} q_{Ai} +p_{5} q_{Bo} q_{Ao} e^{(F V_m/R T)}+p_{6} q_{Bi} q_{Ai}+p_{7} q_{Bi} q_{Ao} e^{(F V_m/R T)}+p_{8} q_{Bi} q_{Bo} q_{Ai}+p_{9} q_{Bi} q_{Bo} q_{Ao} e^{(F V_m/R T)}+p_{10} q_{Bo} q_{Ai} q_{Ao} e^{(F V_m/R T)} +p_{11} q_{Bi} q_{Ai} q_{Ao} e^{(F V_m/R T)}+p_{12} q_{Bi} q_{Bo} q_{Ai} q_{Ao} e^{(F V_m/R T)})}$"


# Section 1: Template selection

In [3]:
# Template Selection and BG structure demonstration

singleSelection = widgets.Dropdown(
            options=['Template 3', 'Template 4', 'Template 7', 'Template 11', 'Template 5'],
            description='Select transporter type:',
            style={'description_width': 'initial','description_height': 'initial'},
            disabled=False,
            layout = Layout(width='700px', height='60px')
        )

## Set the font size using CSS
font_size = '28px'  # Font size in CSS format

# Generate the CSS code to update the font size
custom_css = f"""
<style>
.custom-text-style .widget-label {{
    font-size: {font_size};
    height: initial;
    width: initial;
}}    
.widget-dropdown select {{
    font-size: {font_size};
}}
.widget-dropdown .widget-label {{
    font-size: {font_size};
}}

</style>
"""

button_load = widgets.Button(
                description='Load template',
                tooltip='Load',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='300px', height='60px')
            )
button_load.style.font_family = 'monospace'
button_load.style.font_size = '28px'
output1 = widgets.Output(layout={'border': '1px solid gray'})
outputMathTemp = widgets.Output(layout={'border': '1px solid gray'})

def button_load_clicked(event):
    with output1:
        clear_output()
        templateGraphPlot(singleSelection.value,10,equations)
    with outputMathTemp:
        clear_output()
        
        # Create a plot to display the expression
        fig, ax = plt.subplots(figsize=(6, 1))
        ax.text(0.5, 0.5, equations[singleSelection.value], fontsize=20, ha='center', va='center')
        ax.axis('off')
        plt.show()
        
        

        # # Create a Math object with the expression
        # math_expression = Math(equations[singleSelection.value])
        # # Display the Math object
        # display(math_expression)
        
# Apply the custom CSS styling
singleSelection.add_class('widget-dropdown')
singleSelection.add_class('custom-text-style')


button_load.on_click(button_load_clicked)
vbox_result = widgets.VBox([button_load, output1,outputMathTemp])   

vbox_text = widgets.VBox([singleSelection, vbox_result])
page1 = widgets.HBox([vbox_text])

display(page1)
display(HTML(custom_css))

# Section 2: Model Description

In [4]:
types = ['Template 3', 'Template 4', 'Template 7', 'Template 11', 'Template 5']

valueRequired1 = dict((el,[]) for el in types)
valueRequired1['Template 3']=['Ao','Ai']
valueRequired1['Template 4']=['Ao','Ai','Bo','Bi']
valueRequired1['Template 7']=['Ao','Ai','Bo','Bi', 'Co', 'Ci']
valueRequired1['Template 11']=['Ao','Ai','Bo','Bi','Co','Ci', 'Do', 'Di']
valueRequired1['Template 5']=['Ao','Ai','Bo','Bi']


button_annotate = widgets.Button(
                description='Describe the model!',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='400px', height='60px')
            )
button_annotate.style.font_family = 'monospace'
button_annotate.style.font_size = '28px'
output_annotations= widgets.Output(layout={'height': '100%'})
    


def button_annotate_clicked(event):
    with output_annotations:
        output_annotations.clear_output()
        text00=widgets.HTML(value="<h3><b><font color='maroon'>Annotate the following components as in the loaded template:<b><h3>")
        display(text00)
        selectedAnnotations = {}

        key = singleSelection.value
        selectedAnnotations[key]={}
       
        for element in valueRequired1[singleSelection.value]:
            selectedAnnotations[key]['V_SS']=['isVersionOf$OPB_00592']
            selectedAnnotations[key][element]=['isVersionOf$OPB_00425','entity$CHEBI:15378','isPartOf$fma70022']

            if key == 'Template 3':
                if element == 'Ao':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Ai':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
            if key == 'Template 4':
                if element == 'Ao':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Ai':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                if element == 'Bi':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Bo':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
            if key == 'Template 7':
                if element == 'Ao':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                if element == 'Ai':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Bo':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Bi':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                if element == 'Co':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Ci':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
            if key == 'Template 11':
                if element == 'Ao':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                if element == 'Ai':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Bo':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Bi':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                if element == 'Co':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Ci':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                if element == 'Do':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Di':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
            if key == 'Template 5':
                if element == 'Ao':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Source')
                if element == 'Ai':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Sink')
                if element == 'Bo':
                    selectedAnnotations[key][element].append('hasSourceParticipant$Source')
                if element == 'Bi':
                    selectedAnnotations[key][element].append('hasSinkParticipant$Sink')
                
                
            text=widgets.HTML(value="<h1><b><font color='teal'>q<sub>{}</sub><b><h1>".format(element))

            display(text)
            
            selectedAnnotations = annotate(key,element,selectedAnnotations)
            
            # Save the values in .json file
        with open("./temporary files/selectedAnnotations.json", "w") as outfile:
            json.dump(selectedAnnotations, outfile, indent=4, sort_keys=True, ensure_ascii=False)
    
            
button_annotate.on_click(button_annotate_clicked)
annoat_page = widgets.VBox([button_annotate,output_annotations])
display(annoat_page)


# Section 3: Parameterization

In [9]:
CellVol_button = widgets.Button(description='Display', layout = Layout(width='300px', height='60px'))
CellVol_button.style.font_size = '28px'
CellVol_button.style.button_color = 'skyblue'
CellVol_button.style.font_family = 'monospace'
CellVol_output = widgets.Output()

cellVol = widgets.FloatText(value=1, description='Insert the cell volume: ', style={'description_width': 'initial'}, layout = Layout(width='450px', height='60px'))
cellVol.style.font_family = 'monospace'
# cellUnit = widgets.Dropdown(options=[('femtolitre',1e-15), ('picolitre',1e-12), ('nanolitre',1e-9), ('microlitre',1e-6)],  style={'description_width': 'initial'})
cellUnit = widgets.Dropdown(options=['femtolitre', 'picolitre', 'nanolitre', 'microlitre'],  style={'description_width': 'initial'}, layout = Layout(width='250px', height='60px'))


vbox_result = widgets.HBox([cellVol, cellUnit])
display(vbox_result)
                           
def validate_positive_value(change):
            value = change['new']
            if value < 0:
                change['owner'].value = 0
                
        
cellVol.observe(validate_positive_value, 'value')
                
                
                
def CellVol_button_clicked(event):
    with CellVol_output:
        clear_output()
        
        # formatted_cellVol = float("{:.2f}".format(cellVol.value))* cellUnit.value
        # print('The cell volume is: '+str(formatted_cellVol)+' (Litre)')
        print('The cell volume is: '+str(cellVol.value)+' '+cellUnit.value)


# Apply the custom CSS class to the FloatText widget
cellVol.add_class('custom-text-style') 
        
CellVol_button.on_click(CellVol_button_clicked)
vbox_result = widgets.VBox([CellVol_button, CellVol_output])
display(vbox_result)
display(HTML(custom_css))

In [27]:
# Getting the SS values
types = ['Template 3', 'Template 4', 'Template 7', 'Template 11', 'Template 5']
valueRequired = dict((el,[]) for el in types)

valueRequired['Template 3']=['V_SS','Ao','Ai']
valueRequired['Template 4']=['V_SS','Ao','Ai','Bo','Bi']
valueRequired['Template 7']=['V_SS','Ao','Ai','Bo','Bi', 'Co', 'Ci']
valueRequired['Template 11']=['V_SS','Ao','Ai','Bo','Bi','Co','Ci', 'Do', 'Di']
valueRequired['Template 5']=['V_SS','Ao','Ai','Bo','Bi', 'V_m']



button_SS = widgets.Button(
                description='Steady-state Values',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='400px', height='60px')
            )
button_SS.style.font_family = 'monospace'
button_SS.style.font_size = '28px'

buttonExperimentalData = widgets.Button(description='Experimental Data',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='400px', height='60px'))
buttonExperimentalData.style.font_family = 'monospace'
buttonExperimentalData.style.font_size = '28px'

output_parameterization= widgets.Output(layout={})
output_csv= widgets.Output(layout={})


        
# Tab 1 functionality
def button_SS_clicked(event):
    with output_parameterization:
        clear_output()

        file = open('./temporary files/selectedAnnotations.json')
        annots = json.load(file)
        
        entities = [('H+','CHEBI:15378'),('HCO3-','CHEBI:17544'), ('K+','CHEBI:29103'), ('Na+','CHEBI:29101'), ('Mg2+','CHEBI:18420'), ('Cl-','CHEBI:17996'), ('Ca2+','CHEBI:29108'), ('Fe2+','CHEBI:29033'), ('P','CHEBI:30207')]
        locations = [('Extracellular environment','fma70022'), ('Cytosol of stem cell','fma260697'), ('Cytosol of neuron','fma226054'), ('Cytosol of connective tissue cell','fma260699'),('Cytosol of skeletal muscle fiber','fma226052'),('Cytosol of cardiac myocyte','fma226050'),('Cytosol of striated muscle cell','fma226048'),('Cytosol of smooth muscle cell','fma226046'),('Cytosol of muscle cell','fma226044'), ('Cytosol of hemal cell','fma260695'), ('Cytosol of epithelial cell','fma260691')]

        
        global floatSSget,Description
        floatSSget=[]; Description=[]
        for i,el in enumerate(valueRequired[singleSelection.value]):
            if el == 'V_SS':
                Description.append(widgets.HTML(value='Steady-state flux'))
            elif el == 'V_m':
                Description.append(widgets.HTML(value='Membrane voltage'))
            else:    
                Description.append(widgets.HTML(value=[y[0] for y in entities if y[1] in [x for x in annots[singleSelection.value][el] if 'entity$' in x][0]][0]+' in '+[y[0] for y in locations if y[1] in [x for x in annots[singleSelection.value][el] if 'isPartOf$' in x][0]][0]))
            floatSSget.append(widgets.FloatText(
                    value=0.0,
                    description=['$q_{}$'.format(el) if el not in ['V_SS','V_m'] else el][0]+['  ($u$mol/s)' if el=='V_SS' else '  (mV)' if el=='V_m' else '  ($u$mol)'][0],
                    disabled=False,
                style={'description_width': 'initial'},
                layout = Layout(width='300px', height='60px')
                ))
            for F in floatSSget:
                F.add_class('widget-dropdown')
                F.add_class('custom-text-style')
            
        def validate_positive_value(change):
            value = change['new']
            if value < 0:
                change['owner'].value = 0.0
                
        for i,el in enumerate(valueRequired[singleSelection.value]):
            if el != 'V_m':
                floatSSget[i].observe(validate_positive_value, 'value')
            
            
        button_saveSS = widgets.Button(
                        description='Save',
                        tooltip='Save',
                        style={'description_width': 'initial'},
                        layout = Layout(width='120px', height='40px'))
        button_saveSS.style.font_family = 'monospace'
        button_saveSS.style.font_size = '24px'
        button_saveSS.style.button_color = 'lightgreen'


        def button_saveSS_clicked(event):
            with output_parameterization:
                # clear_output()
                text = widgets.HTML(value="<h4><b>The inserted steady-state values have been saved.<b><h4>")
                display(text)      
                
                for floatSS in floatSSget:
                    if floatSS.description!='V_m  (mV)':
                        floatSS.value = copy.copy(floatSS.value/cellVol.value)
                        
 
                    

                pVals = pValsFunc(singleSelection,floatSSget,dataset=False)
                # Replace the parameter values in the equation
                equation = copy.deepcopy(equations[singleSelection.value])
                for param, value in pVals[singleSelection.value].items():
                    param_pattern = re.escape('p_{' + param[1:] + '}')
                    if abs(value) >= 0.01:
                        formatted_value = "{:.2f}".format(value)
                    else:
                        formatted_value = "{:.2e}".format(value)
                    equation = re.sub(param_pattern, formatted_value, equation)
                    
                # Create a plot to display the expression
                fig, ax = plt.subplots(figsize=(6, 1))
                ax.text(0.5, 0.5, equation, fontsize=20, ha='center', va='center')
                ax.axis('off')
                plt.show()


        button_saveSS.on_click(button_saveSS_clicked)
        
        for i in range(len(Description)):
            display(Description[i],floatSSget[i])
            display(HTML(custom_css))

        display(button_saveSS)
        display(HTML(custom_css))
        
        
# Tab 2 functionality
def buttonExperimentalData_clicked(event):
    with output_csv:
        clear_output()        
        # Create widgets for the second tab
        file_upload_widget = widgets.FileUpload(accept='.csv')
        submit_button_widget = widgets.Button(description='Submit')
        


        def submit_button_clicked(event):
            with output_csv:
                clear_output()
                # Get the uploaded file
                uploaded_files = file_upload_widget.value[0]
                if uploaded_files:
                    file_contents = uploaded_files['content']
                    df = pd.read_csv(io.BytesIO(file_contents))
                    
###############################################################################################################                    
                    
                    file = open('./temporary files/selectedAnnotations.json')
                    annots = json.load(file)

                    entities = [('H+','CHEBI:15378'),('HCO3-','CHEBI:17544'), ('K+','CHEBI:29103'), ('Na+','CHEBI:29101'), ('Mg2+','CHEBI:18420'), ('Cl-','CHEBI:17996'), ('Ca2+','CHEBI:29108'), ('Fe2+','CHEBI:29033'), ('P','CHEBI:30207')]
                    locations = [('Extracellular environment','fma70022'), ('Cytosol of stem cell','fma260697'), ('Cytosol of neuron','fma226054'), ('Cytosol of connective tissue cell','fma260699'),('Cytosol of skeletal muscle fiber','fma226052'),('Cytosol of cardiac myocyte','fma226050'),('Cytosol of striated muscle cell','fma226048'),('Cytosol of smooth muscle cell','fma226046'),('Cytosol of muscle cell','fma226044'), ('Cytosol of hemal cell','fma260695'), ('Cytosol of epithelial cell','fma260691')]


                    Description=[]
                    for i,el in enumerate(valueRequired[singleSelection.value]):
                        if el == 'V_SS':
                            Description.append(widgets.HTML(value='Steady-state flux'))
                        elif el == 'V_m':
                            Description.append(widgets.HTML(value='Membrane voltage'))
                        else:    
                            Description.append(widgets.HTML(value=[y[0] for y in entities if y[1] in [x for x in annots[singleSelection.value][el] if 'entity$' in x][0]][0]+' in '+[y[0] for y in locations if y[1] in [x for x in annots[singleSelection.value][el] if 'isPartOf$' in x][0]][0]))
   
                    
                    # List of new headers
                    new_headers = copy.deepcopy(valueRequired[singleSelection.value])

                    # Dropdown widgets to select headers
                    dropdowns = []
                    selected_headers = {}

                    def create_dropdown(header):
                        dropdown = widgets.Dropdown(
                            options=[''] + df.columns.tolist(),
                            description=header + ':',
                            layout = Layout(width='250px', height='60px'),
                            style={'description_width': 'initial'},
                            disables=False)
                        
                        # Apply the custom CSS class to the FloatText widget
                        dropdown.add_class('widget-dropdown')
                        dropdown.add_class('custom-text-style')
                        display(HTML(custom_css))
                        
                        dropdowns.append(dropdown)

                        def dropdown_event(change, header=header):
                            selected_headers[header] = change.new

                        dropdown.observe(dropdown_event, names='value')

                    # Create dropdowns for each header
                    for header in new_headers:
                        create_dropdown(['$q_{}$'.format(header) if header not in ['V_SS','V_m'] else header][0]+['  ($u$mol/s)' if header=='V_SS' else '  (mV)' if header=='V_m' else '  ($u$mol)'][0])
                        display(HTML(custom_css))

                    # Button to show dropdowns
                    show_button = widgets.Button(description='Label Selection', layout = Layout(width='300px', height='60px'))
                    show_button.style.font_family = 'monospace'
                    show_button.style.font_size = '28px'
                    show_button.style.button_color = 'lightblue'
                    
                    dropdown_output = widgets.Output()

                    def show_button_clicked(button):
                        with dropdown_output:
                            dropdown_output.clear_output()
                            for i in range(len(Description)):
                                display(Description[i],dropdowns[i])
                                display(HTML(custom_css))
                            generate_button.layout.visibility = 'visible'

                    show_button.on_click(show_button_clicked)

                    # Button to generate new dataframe
                    generate_button = widgets.Button(description='Display the Steady-state Equation', layout = Layout(width='600px', height='60px'))
                    generate_button.style.font_size = '28px'
                    generate_button.style.font_family = 'monospace'
                    generate_button.style.button_color = 'skyblue'
                    generate_button.layout.visibility = 'hidden'  # Initially hidden
                    generate_output = widgets.Output()

                    def generate_button_clicked(button):
                        with generate_output:
                            generate_output.clear_output()
                            selected_values = list(selected_headers.values())
                            if len(set(selected_values)) == len(new_headers) and all(selected_values):
                                if len(set(selected_values)) == len(selected_values):
                                    new_data = {['$q_{}$'.format(new_header) if new_header not in ['V_SS','V_m'] else new_header][0]+['  ($u$mol/s)' if new_header=='V_SS' else '  (mV)' if new_header=='V_m' else '  ($u$mol)'][0]: df[selected_headers[['$q_{}$'.format(new_header) if new_header not in ['V_SS','V_m'] else new_header][0]+['  ($u$mol/s)' if new_header=='V_SS' else '  (mV)' if new_header=='V_m' else '  ($u$mol)'][0]]] for new_header in new_headers}
                                    # global new_df
                                    new_df = pd.DataFrame(new_data)
                                    
                                    # global floatSSget
                                    floatSSget=[]
                                    for header in new_df:
                                        floatSSget.append(widgets.FloatText(
                                                            value=new_df[header].iloc[-1],
                                                            description=header,
                                                            disabled=False,
                                                             layout = Layout(width='600px', height='60px')))
                                        
                                        
                                    for floatSS in floatSSget:
                                        if floatSS.description!='V_m  (mV)':
                                            floatSS.value = copy.copy(floatSS.value/cellVol.value)
                                            
                                    for F in floatSSget:
                                        F.add_class('widget-dropdown')
                                        F.add_class('custom-text-style')

                                        # display(HTML(custom_css))
                                    
                                    pVals = pValsFunc(singleSelection,new_df,dataset=True)
                                    # Replace the parameter values in the equation
                                    equation = copy.deepcopy(equations[singleSelection.value])
                                    for param, value in pVals[singleSelection.value].items():
                                        param_pattern = re.escape('p_{' + param[1:] + '}')
                                        if abs(value) >= 0.01:
                                            formatted_value = "{:.2f}".format(value)
                                        else:
                                            formatted_value = "{:.2e}".format(value)
                                        equation = re.sub(param_pattern, formatted_value, equation)

                                    # Create a plot to display the expression
                                    fig, ax = plt.subplots(figsize=(6, 1))
                                    ax.text(1, 1, equation, fontsize=28, ha='center', va='center')
                                    ax.axis('off')
                                    plt.show()
                                else:
                                    print("Please select unique headers for all lists.")
                            else:
                                print("Please select headers for all lists (Check your selected data to be valid)")

                    generate_button.on_click(generate_button_clicked)

                    # Display buttons, dropdowns, and new dataframe output
                    display(show_button, dropdown_output, generate_button, generate_output)
                    display(HTML(custom_css))


#################################################
                    
                    
                else:
                    print('Please upload a CSV file.')

        submit_button_widget.on_click(submit_button_clicked)
        vbox_result = widgets.VBox([file_upload_widget, submit_button_widget])
        display(vbox_result)
        
        

button_SS.on_click(button_SS_clicked)
buttonExperimentalData.on_click(buttonExperimentalData_clicked)

# Create the tab widgets
SS_page = widgets.VBox([button_SS, output_parameterization])
ED_page = widgets.VBox([buttonExperimentalData, output_csv])
display(HTML(custom_css))

# Create the Tab widget
tab = widgets.Tab()
tab.children = [SS_page, ED_page]
tab.set_title(0, "Option 1")
tab.set_title(1, "Option 2")

# Create a callback to clear the output when switching tabs
def clear_output_callback(change):
    output_parameterization.clear_output()
    output_csv.clear_output()
    
# Assign the callback to the selected_index attribute of the Tab widget
tab.observe(clear_output_callback, 'selected_index')

# Display the Tab widget and output
display(tab)
display(HTML(custom_css))


# Section 4: Stimulus

In [29]:

button_stim = widgets.Button(
                description='Run',
                tooltip='Description',
                style={'description_width': 'initial'},
                button_style='warning',
                layout = Layout(width='120px', height='40px'))
button_stim.style.font_family = 'monospace'
button_stim.style.font_size = '28px'
outputStim = widgets.Output(layout={'border': '1px solid gray'})


def stimulate(event):
    with outputStim:
        outputStim.clear_output()
        img = Image.open('./figures/stimuli/step.jpg')
        display(img.resize((200, 200)))
        
        V_m=[]
        if singleSelection.value == 'Template 5':
            V_m = [x.value for x in floatSSget if x.description=='V_m'+'  (mV)']
            

        t_eval = np.arange(0,100,0.1)

        
        # Create radio buttons and slider
        global control_rb, control_slider, sliders
        
        control_rb = widgets.RadioButtons(
            options=[('q_{}'.format(x), x) for x in valueRequired1[singleSelection.value]]+[(x,x) for x in ['V_m'] if V_m!=[]], 
            description='Control variable:',
            style={'description_width': 'initial'},
        layout=Layout(width='100%'))
        
        control_slider = widgets.FloatSlider(
            value=[x.value for x in floatSSget if (x.description=='$q_{}$'.format(control_rb.value)+'  ($u$mol)' or x.description=='V_m'+'  (mV)')][0], min=-200, max=1000, step=0.001, 
            description='Control value:',
            style={'description_width': 'initial'},
        layout=Layout(width='100%'))
        
        sliders = {}
        for i,species in enumerate(valueRequired1[singleSelection.value]+[x for x in ['V_m'] if V_m!=[]]):
            sliders[species] = widgets.FloatSlider(
            value = [x.value for x in floatSSget if x.description=='$q_{}$'.format(species)+'  ($u$mol)' or x.description=='V_m  (mV)'][0], min=-200, max=1000, step=0.001, 
            description=[x.description for x in floatSSget if x.description=='$q_{}$'.format(species)+'  ($u$mol)' or x.description=='V_m  (mV)'][0]+' :',
            layout=Layout(width='50%'))

        stepTime=FloatSlider(value=1000,min=0,max=1000,step=0.1,layout=Layout(width='100%'), readout_format='.3f', style={'description_width': 'initial'}, description='t0')        


        # Interact with the update function to update the plot
        widgets.interact(update_figure,
                         control_variable=control_rb, 
                         control_value=control_slider,
                         stepTime=stepTime,
                         timespan=widgets.FloatSlider(value=1000, min=0, max=1000, step=0.1, description='Timespan:'),
                         **sliders
                         )
        
        # Save the selected conditions
        button_saveControl = widgets.Button(
                        description='Save',
                        tooltip='Save',
                        style={'description_width': 'initial'},
                        layout = Layout(width='120px', height='40px'))
        button_saveControl.style.font_family = 'monospace'
        button_saveControl.style.font_size = '20px'
        button_saveControl.style.button_color = 'lightgreen'
        output_saveControl=widgets.Output(layout={})


        def button_saveControl_clicked(event):
            with output_saveControl:
                clear_output()
                textControl = widgets.HTML(value="<h4><b>The selected control value has been saved.<b><h4>")
                display(textControl)      
                for floatSS in floatSSget:
                    if  '$q_{}$'.format(control_rb.value)+'  ($u$mol)'== floatSS.description:
                        floatSS.value = copy.copy(control_slider.value)

        button_saveControl.on_click(button_saveControl_clicked)
        vbox_result = widgets.VBox([button_saveControl, output_saveControl]) 
        pageSaveControl = widgets.HBox([vbox_result])
        display(pageSaveControl)        
        
        
button_stim.on_click(stimulate)
vbox_result = widgets.VBox([button_stim, outputStim]) 
page7 = widgets.HBox([vbox_result])
display(page7)

# Section 5: Save the Model

In [36]:
button_modelName = widgets.Textarea(
                description='Give it a name:',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout = Layout(width='400px', height='50px'))

button_modelName.style.font_family = 'monospace'
button_modelName.style.font_size = '28px'
button_modelName.style.button_color = 'lightgreen'
button_modelName.add_class('widget-dropdown')
button_modelName.add_class('custom-text-style')
display(button_modelName)


button_addModel = widgets.Button(
                description='Save the model',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout = Layout(width='250px', height='40px'))
button_addModel.style.font_family = 'monospace'
button_addModel.style.font_size = '28px'
button_addModel.style.button_color = 'lightgreen'
output9 = widgets.Output(layout={})

def button_addModel_clicked(event):
    with output9:
        output9.clear_output()
        if button_modelName.value.replace('\n','') not in savedModels and button_modelName.value.replace('\n','')!='' and button_modelName.value.replace(' ','') not in savedModels:
            savedModels.append(button_modelName.value.replace('\n',''))
            text = widgets.HTML(value="<h3>Your generated model has been saved.</h3>")
            display(text)
            # Save the annotations as a new json file
            f = open("./temporary files/selectedAnnotations.json")
            file = json.load(f)
            file[button_modelName.value.replace('\n','')]=file[singleSelection.value]
            del file[singleSelection.value]
            savedAnnotations.update(file)
            with open('./savedAnnotations.json', 'w') as f:
                json.dump(savedAnnotations, f, indent=4, sort_keys=True, ensure_ascii=False)  
                
            # Save the P values as a new json file
            f = open("./temporary files/pVals.json")
            file = json.load(f)
            file[button_modelName.value.replace('\n','')]=file[singleSelection.value]
            del file[singleSelection.value]
            savedpVals.update(file)
            with open('./savedModels/savedpVals.json', 'w') as f:
                json.dump(savedpVals, f, indent=4, sort_keys=True, ensure_ascii=False)  
            
            # Save the type as a new json file    
            file=dict([(button_modelName.value.replace('\n',''),singleSelection.value)])
            savedtypes.update(file)
            with open('./savedModels/savedtypes.json', 'w') as f:
                json.dump(savedtypes, f, indent=4, sort_keys=True, ensure_ascii=False)
                
            
        elif button_modelName.value.replace('\n','') == '':   
            
            text = widgets.HTML(value="<h3><font color='red'>None: Enter a valid name.</h3>")
            display(text)
        else:
            text = widgets.HTML(value="<h3><font color='red'>Repetitive name: Enter a new name.</h3>")
            display(text)
        
        print('Your saved model is:')
        modelUnits = {'Concentration': 'micro', 'Flow rate': 'milli'}
        model = singleModelCellmlGenerator(modelUnits,savedpVals,button_modelName.value)
        xmlAnnot(model,savedAnnotations)
        
button_addModel.on_click(button_addModel_clicked)
vbox_result = widgets.VBox([button_addModel, output9]) 
display(vbox_result)

Textarea(value='', description='Give it a name:', layout=Layout(height='50px', width='400px'), style=TextStyle…

<div style="text-align: center;">
    <h3>👉🏻 Upload the saved model on a PMR workspace 👈🏻</h3>
    <img src="./figures/physiome.png" alt="Figure" width="200" height="200">
</div>

<div class="alert alert-block alert-warning">
    <h1><b>Model Composition</b></h1> 
</div>

***

# Section 1: Search on PMR

In [41]:
search_modelName = widgets.Textarea(
                description='Enter a model name:',
                tooltip='Description',
                style={'description_width': 'initial'},
    layout = Layout(width='450px', height='50px')
)
search_modelName.style.font_family = 'monospace'
search_modelName.style.font_size = '28px'
search_modelName.style.button_color = 'red'
search_modelName.add_class('widget-dropdown')
search_modelName.add_class('custom-text-style')
display(search_modelName)

button_PMR = widgets.Button(
                description='Search on PMR',
                tooltip='Search',
                style={'description_width': 'initial'},
                button_style='info',
                layout = Layout(width='300px', height='40px'))

button_PMR.style.font_family = 'monospace'
button_PMR.style.font_size = '28px'
button_PMR.style.button_color = 'red'
button_PMR.add_class('widget-dropdown')
button_PMR.add_class('custom-text-style')
outputPMR = widgets.Output()

global downloadedModels,xmlParticles, xmlGrouped
downloadedModels=[]
xmlParticles={}; xmlGrouped={}

dir_name = "./savedModels/PMR"
test = os.listdir(dir_name)

for item in test:
    if item.endswith(".cellml"):
        os.remove(os.path.join(dir_name, item))
        
        
    
def button_PMR_clicked(event):
    with outputPMR:
        clear_output()
        
        pmrModel = pmrSearching(search_modelName.value.replace(' ','%20'))
        
        xmlParticles = RDFpmrSearching(search_modelName.value.replace(' ','%20'))
        
        buttons=[]
        links=[]
        outputs = []
        
        if pmrModel !=[]:
            for i,el in enumerate(pmrModel):
                links.append(el)

            for link in links:
                text=widgets.HTML(value=link)
                button = widgets.Button(description='Select',tooltip='Select',style={'description_width': 'initial'},button_style='info')
                output = widgets.Output()
                display(widgets.HBox([text,button]))
                display(output)
                buttons.append(button)
                outputs.append(output)    

            def on_button_click(button):
                with outputs[buttons.index(button)]:
                    clear_output()
                    text=widgets.HTML("<h3><font color='red'>Selected!</h3>")
                    display(text)
                    filename = os.path.basename(links[buttons.index(button)]).split('.')[0]
                    folder_path = './savedModels/PMR'
                    # Check if the folder exists, and create it if it doesn't
                    if not os.path.exists(folder_path):
                        os.makedirs(folder_path)
    
                    file = "./savedModels/PMR/{}.cellml".format(filename)
                    urllib.request.urlretrieve(links[buttons.index(button)], file)
                    downloadedModels.append(filename+'.cellml')
                    
                    for workspace in xmlParticles:
                        for ID,predicate,obj in xmlParticles[workspace]:
                            if workspace in links[buttons.index(button)] and ID.split('#')[0] in links[buttons.index(button)]:
                                if ID.split('#')[0] not in xmlGrouped:
                                    xmlGrouped[ID.split('#')[0]]={} #model name
                                    if ID.split('#')[1] not in  xmlGrouped[ID.split('#')[0]]:
                                        xmlGrouped[ID.split('#')[0]][ID.split('#')[1]]=[]
                                        xmlGrouped[ID.split('#')[0]][ID.split('#')[1]].append((predicate,obj))
                                    else:
                                        xmlGrouped[ID.split('#')[0]][ID.split('#')[1]].append((predicate,obj))
                                else:
                                    if ID.split('#')[1] not in  xmlGrouped[ID.split('#')[0]]:
                                        xmlGrouped[ID.split('#')[0]][ID.split('#')[1]]=[]
                                        xmlGrouped[ID.split('#')[0]][ID.split('#')[1]].append((predicate,obj))
                                    else:
                                        
                                        xmlGrouped[ID.split('#')[0]][ID.split('#')[1]].append((predicate,obj))
                                        
                    

            for button in buttons:
                button.on_click(lambda b: on_button_click(b))
                
                
        else:
                print('No such a file on PMR')

        
            
button_PMR.on_click(button_PMR_clicked)
vbox_result = widgets.VBox([button_PMR, outputPMR])   
display(vbox_result)

Textarea(value='', description='Enter a model name:', layout=Layout(height='50px', width='450px'), style=TextS…

# Section 2: Model Composition

In [42]:
button_compose = widgets.Button(
                description='Compose the models',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout = Layout(width='300px', height='40px'))
button_compose.style.font_family = 'monospace'
button_compose.style.font_size = '28px'
button_compose.style.button_color = 'orange'
compositionOutput = widgets.Output(layout={})

def button_compose_clicked(event):
    with compositionOutput:
        compositionOutput.clear_output()


        PMRmodelComposition()

        file_name = 'CompositeModel'
        pythonCreator('./',file_name)

        %run {file_name}.py



##############################################################################################################################################
        my_rates = np.zeros((STATE_COUNT))
        my_variables=np.zeros((VARIABLE_COUNT))
        my_state_variables=np.zeros((STATE_COUNT))

        initialise_variables(my_state_variables, my_variables)
        
        # Create sliders
        global control_slider
        control_slider = {}
        for i,species in enumerate(speciesNoDuplicate):
                    control_slider[species[-1]] = widgets.FloatSlider(
                    value = my_state_variables[i], min=0, max=[my_state_variables[i]*10 if my_state_variables[i]!=0 else 10][0], step=0.001, 
                    description=species[-1]+' :',
                    layout=Layout(width='50%'))

        stepSize=widgets.Dropdown(options=[0.001,0.01,0.1,1.0,10,100],
                                  value=0.001,
                                  description='Step size:',
                                  style={'description_width': 'initial'},
                                  disabled=False)
        
        # timespan=widgets.FloatSlider(value=10, min=0, max=100000, step=stepSize.value, description='Timespan (ms) :', style={'description_width': 'initial'}, layout=Layout(width='50%'))
        timespan_text = widgets.FloatText(value=10, style={'description_width': 'initial'}, description='Timespan (ms):')

        
        # Define the update function
        def update_composed_figure(T, stepSize, **control_value):
             
            my_rates = np.zeros((STATE_COUNT))
            my_variables=np.zeros((VARIABLE_COUNT))
            my_state_variables=np.zeros((STATE_COUNT))

            initialise_variables(my_state_variables, my_variables)
            for i,species in enumerate(speciesNoDuplicate):
                my_state_variables[i] = control_value[species[-1]]
            compute_computed_constants(my_variables)
            compute_rates('t', my_state_variables, my_rates, my_variables)

            global Solution
            step_size = stepSize   
            step_count = int(T/step_size)  
            Solution = {}

            for step in range(0, step_count):
                time = step * step_size

                Solution[step] = []
                Solution[step].append(time)

                # Compute the rates at this step using the given function.
                compute_rates(time, my_state_variables, my_rates, my_variables)

                # Compute the states.                
                for s in range(0, STATE_COUNT):
                    my_state_variables[s] = my_state_variables[s] + my_rates[s] * step_size

                # Compute the variables.
                compute_variables(time, my_state_variables, my_rates, my_variables)


                for s in range(0, VARIABLE_COUNT):
                    Solution[step].append(my_variables[s])

                for s in range(0, STATE_COUNT):
                    Solution[step].append(my_state_variables[s])



            # Update the plot
            entities = [('H+','CHEBI:15378'),('HCO3-','CHEBI:17544'), ('K+','CHEBI:29103'), ('Na+','CHEBI:29101'), ('Mg2+','CHEBI:18420'), ('Cl-','CHEBI:17996'), ('Ca2+','CHEBI:29108'), ('Fe2+','CHEBI:29033'), ('P','CHEBI:30207')]
            locations = [('Extracellular environment','fma70022'), ('Cytosol of stem cell','fma260697'), ('Cytosol of neuron','fma226054'), ('Cytosol of connective tissue cell','fma260699'),('Cytosol of skeletal muscle fiber','fma226052'),('Cytosol of cardiac myocyte','fma226050'),('Cytosol of striated muscle cell','fma226048'),('Cytosol of smooth muscle cell','fma226046'),('Cytosol of muscle cell','fma226044'), ('Cytosol of hemal cell','fma260695'), ('Cytosol of epithelial cell','fma260691')]

            matplotlib.rcParams.update({'font.size': 15})
            fig, ax = plt.subplots(nrows=[int(len(speciesNoDuplicate)/2) if len(speciesNoDuplicate)%2==0 else int(len(speciesNoDuplicate)/2)+1][0],ncols=2,figsize=(25,7*len(speciesNoDuplicate)/2))
            plt.subplots_adjust(
                    wspace=0.2,
                    hspace=0.4)
            for i,species in enumerate(speciesNoDuplicate):
                line = []
                timecourse=[]

                for step in Solution:
                    timecourse.append(list(Solution[step])[0])
                    line.append(list(Solution[step])[-(len(speciesNoDuplicate)-i)])
                    
                    
                if i < [int(len(speciesNoDuplicate)/2) if len(speciesNoDuplicate)%2==0 else int(len(speciesNoDuplicate)/2)+1][0]:
                    k=copy.copy(i)
                    j=0
                else:
                    k=copy.copy(i-[int(len(speciesNoDuplicate)/2) if len(speciesNoDuplicate)%2==0 else int(len(speciesNoDuplicate)/2)+1][0])
                    j=1

                ax[k,j].plot(timecourse, line, color=matplotlib.cm.hsv(float(i) / len(speciesNoDuplicate)), linewidth=5, label=species[-1])
                ax[k,j].set_xlabel('Time (s)')
                ax[k,j].grid(color='gray', linestyle='-', linewidth=0.5)
                ax[k,j].set_ylabel('Amount ($u$mol)')
                ax[k,j].legend(loc='best')
                ax[k,j].patch.set_facecolor('black')


                for x in species:
                    for y in locations:
                        if x[0]=='http://biomodels.net/biology-qualifiers/isPartOf' and y[1] in x[1]:
                            location = [y[0]]
                    for y in entities:
                        if y[1] in x[1] and x[0]=='http://biomodels.net/biology-qualifiers/isVersionOf' and 'https://identifiers.org/opb' not in x[1]:
                            entity=[y[0]]


                if entity!=[] and location!=[]:
                    title = entity+[' in ']+location
                if entity==[] and location!=[]:
                    title = ['unidentified entity in ']+location
                if location==[] and entity!=[]:
                    title = entity+[' in unidentified location']
                if entity==[] and location==[]:
                    title = ['unidentified entity']+[' in unidentified location']
                ax[k,j].set_title(''.join(title))
                    
                    
                    
            plt.show()


        # Interact with the update function to update the plot
        widgets.interact(update_composed_figure, 
                         T=timespan_text,
                         stepSize=stepSize,
                         **control_slider
                         )     
        
        button_saveSS = widgets.Button(
                        description='Save',
                        tooltip='Save',
                        style={'description_width': 'initial'},
                        layout = Layout(width='120px', height='40px'))
        button_saveSS.style.font_family = 'monospace'
        button_saveSS.style.font_size = '20px'
        button_saveSS.style.button_color = 'lightgreen'
        outputInteract = widgets.Output(layout={})

        def button_saveSS_clicked(event):
            with outputInteract:
                clear_output()
                text = widgets.HTML(value="<h3><b>The model is updated with the selected values.<b><h3>")
                display(text)  
                modelUpdate = parse_model(os.path.join('./', 'CompositeModel.cellml'), False)
                for key in control_slider.keys():
                    for compNum in range(modelUpdate.componentCount()):
                        for varNum in range(modelUpdate.component(compNum).variableCount()):
                            if modelUpdate.component(compNum).variable(varNum).name()==key:
                                modelUpdate.component(compNum).variable(varNum).setInitialValue(control_slider[key].value)
                printer = libcellml.Printer()
                # print(printer.printModel(modelUpdate))
                writeCellML(modelUpdate,printer,'CompositeModel')

        button_saveSS.on_click(button_saveSS_clicked)

        vbox_result = widgets.VBox([button_saveSS, outputInteract])   
        pageInteract = widgets.HBox([vbox_result])
        display(pageInteract)
                        
##########################################################################################################################################################                        
          
            
            
button_compose.on_click(button_compose_clicked)
vbox_result = widgets.VBox([button_compose, compositionOutput]) 
display(vbox_result)

# Section 3: Graph Representation

In [50]:
%run function_imports.ipynb
button_graph = widgets.Button(
                description='Plot the graph',
                tooltip='Description',
                style={'description_width': 'initial'},
                layout = Layout(width='300px', height='40px'))
button_graph.style.font_family = 'monospace'
button_graph.style.font_size = '28px'
button_graph.style.button_color = 'gray'
button_graph.add_class('widget-dropdown')
button_graph.add_class('custom-text-style')
graphOutput = widgets.Output(layout={})

figSize = widgets.FloatText(value=20, description='Figure size: ', layout = Layout(width='250px', height='40px'))
figSize.add_class('widget-dropdown')
figSize.add_class('custom-text-style')

display(figSize)
def button_graph_clicked(event):
    with graphOutput:
        graphOutput.clear_output()

        graphPlot(newCellmlNames,Species,speciesNoDuplicate,figSize.value)
        
        
        
        
button_graph.on_click(button_graph_clicked)
vbox_result = widgets.VBox([button_graph, graphOutput]) 
display(vbox_result)

FloatText(value=20.0, description='Figure size: ', layout=Layout(height='40px', width='250px'), _dom_classes=(…